## CuDNNLSTM
98.8% accuracy

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, CuDNNLSTM
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist

# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# resize and normalize
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size])
x_test = np.reshape(x_test,[-1, image_size, image_size])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (image_size, image_size)
batch_size = 128
units = 256
dropout = 0.2

# model is RNN with 256 units, input is 28-dim vector 28 timesteps
model = Sequential()
model.add(CuDNNLSTM(units=units,
                    input_shape=input_shape))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='cudnnlstm-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of adam optimizer (better than sgd)
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# train the network
model.fit(x_train, y_train, epochs=20, batch_size=batch_size)

loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_2 (CuDNNLSTM)     (None, 256)               292864    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 295,434
Trainable params: 295,434
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
60000/60000 [==============================] - 5s 80us/step - loss: 0.4171 - acc: 0.8623
Epoch 2/20
60000/60000 [==============================] - 5s 78us/step - loss: 0.1282 - acc: 0.9605
Epoch 3/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0842 - acc: 0.9737
Epoch 4/20
60000/60000 [==============================] - 5s 85us/step - loss: 0.0636 - acc: 0.9801
